# Import the Dataset

In [2]:
#importing the required modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sympy import symbols, exp, Mul , pi, sqrt, DiracDelta
import seaborn as sns

from sympy.utilities.lambdify import lambdify
import pandas as pd
import csv
from scipy.stats import norm
import math

from google.colab import drive

In [3]:
#shiham

drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive/Kaggle


Mounted at /content/gdrive/
/content/gdrive/MyDrive/Kaggle


In [4]:
csv_file = 'weather_data_processed.csv'#make sure to have the Labels column
# csv_file = 'csv_all.csv'
df = pd.read_csv(csv_file)


In [ ]:
dataset.nunique()

day_id                 479
temperature_celsius     81
condition_text           9
wind_kph                56
wind_degree            134
                      ... 
city_id_C108             2
city_id_C109             2
city_id_C110             2
city_id_C111             2
city_id_C112             2
Length: 128, dtype: int64

# Check For outliers

In [5]:
df['condition_text'].value_counts()  # No outliers

condition_text
0    122
1    122
2     68
3     53
4     50
5     21
6     16
7     15
8     12
Name: count, dtype: int64

In [ ]:
# Convert the time columns to datetime
daily_data['sunrise'] = pd.to_datetime(daily_data['sunrise'], format='%I:%M %p')
daily_data['sunset'] = pd.to_datetime(daily_data['sunset'], format='%I:%M %p')

# Convert Clock data to minutes
daily_data['sunrise'] = daily_data['sunrise'].dt.hour * 60 + daily_data['sunrise'].dt.minute
daily_data['sunset'] = daily_data['sunset'].dt.hour * 60 + daily_data['sunset'].dt.minute
daily_data

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,364,1159
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,365,1158
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,365,1158
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,366,1156
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,367,1155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,C112,17.1,Clear and Sunny,13.3,61,1017.0,0.0,57,3,17.1,10.0,1.0,22.2,1,349,1071
2889,D2890,C112,17.4,NaN,13.0,51,1017.0,0.0,49,0,17.4,10.0,1.0,22.2,1,349,1072
2890,D2891,C112,19.2,NaN,11.5,46,1016.0,0.0,34,0,19.2,10.0,1.0,21.3,2,348,1072
2891,D2892,C112,19.2,NaN,14.4,76,1017.0,0.0,45,2,19.2,10.0,1.0,24.9,2,347,1072


In [ ]:
# One-hot encoding the 'city_id' column
daily_data = pd.get_dummies(daily_data, columns=['city_id'])
daily_data

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
0,D0001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,...,False,False,False,False,False,False,False,False,False,False
1,D0002,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,...,False,False,False,False,False,False,False,False,False,False
2,D0003,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,...,False,False,False,False,False,False,False,False,False,False
3,D0004,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,...,False,False,False,False,False,False,False,False,False,False
4,D0005,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,Clear and Sunny,13.3,61,1017.0,0.0,57,3,17.1,...,False,False,False,False,False,False,False,False,False,True
2889,D2890,17.4,NaN,13.0,51,1017.0,0.0,49,0,17.4,...,False,False,False,False,False,False,False,False,False,True
2890,D2891,19.2,NaN,11.5,46,1016.0,0.0,34,0,19.2,...,False,False,False,False,False,False,False,False,False,True
2891,D2892,19.2,NaN,14.4,76,1017.0,0.0,45,2,19.2,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
daily_data['city_id_C109'].dtypes

dtype('bool')

In [ ]:
# Convert all boolean columns to integers (1, 0)
daily_data = daily_data.astype({col: 'int' for col in daily_data.select_dtypes(include='bool').columns})
daily_data.dtypes

day_id                  object
temperature_celsius    float64
condition_text          object
wind_kph               float64
wind_degree              int64
                        ...   
city_id_C108             int32
city_id_C109             int32
city_id_C110             int32
city_id_C111             int32
city_id_C112             int32
Length: 128, dtype: object

In [ ]:
daily_data

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
0,D0001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,...,0,0,0,0,0,0,0,0,0,0
1,D0002,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,...,0,0,0,0,0,0,0,0,0,0
2,D0003,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,...,0,0,0,0,0,0,0,0,0,0
3,D0004,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,...,0,0,0,0,0,0,0,0,0,0
4,D0005,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,Clear and Sunny,13.3,61,1017.0,0.0,57,3,17.1,...,0,0,0,0,0,0,0,0,0,1
2889,D2890,17.4,NaN,13.0,51,1017.0,0.0,49,0,17.4,...,0,0,0,0,0,0,0,0,0,1
2890,D2891,19.2,NaN,11.5,46,1016.0,0.0,34,0,19.2,...,0,0,0,0,0,0,0,0,0,1
2891,D2892,19.2,NaN,14.4,76,1017.0,0.0,45,2,19.2,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
#Save file
daily_data.to_csv('daily_data.csv_processed.csv', index=False)

# Create the training set

In [ ]:
df = daily_data.copy()
df = df.dropna(subset=['condition_text'])
df

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
2,D0003,20.0,Light Rain with Thunder,3.6,10,1011.0,4.50,100,75,20.0,...,0,0,0,0,0,0,0,0,0,0
3,D0004,17.0,Clear and Sunny,6.1,150,1018.0,0.00,88,0,17.0,...,0,0,0,0,0,0,0,0,0,0
6,D0007,21.0,Partly Cloudy,4.0,310,1015.0,0.00,100,50,21.0,...,0,0,0,0,0,0,0,0,0,0
18,D0019,19.0,Clear and Sunny,3.6,64,1017.0,0.00,88,0,19.0,...,0,0,0,0,0,0,0,0,0,0
27,D0028,19.0,Partly Cloudy,3.6,83,1010.0,0.00,73,25,19.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,D2839,27.0,Partly Cloudy,19.1,90,1014.0,0.01,70,25,29.8,...,0,0,0,0,0,0,0,1,0,0
2863,D2864,21.4,Light Precipitation,3.6,178,1012.0,0.00,89,75,21.4,...,0,0,0,0,0,0,0,0,1,0
2873,D2874,16.9,Clear and Sunny,16.6,68,1018.0,0.00,44,2,16.9,...,0,0,0,0,0,0,0,0,0,1
2886,D2887,19.5,Clear and Sunny,5.4,27,1014.0,0.00,34,0,19.4,...,0,0,0,0,0,0,0,0,0,1


# Encode The labels

In [ ]:
weather_mapping = {
    'Clear and Sunny': 0,
    'Partly Cloudy': 1,
    'Light Precipitation': 2,
    'Cloudy and Overcast': 3,
    'Mist or Fog': 4,
    'Rain Showers': 5,
    'Light Rain with Thunder': 6,
    'Thunderstorms': 7,
    'Moderate to Heavy Rain': 8
}

In [ ]:
df['condition_text'] = df['condition_text'].map(weather_mapping)
df

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
2,D0003,20.0,6,3.6,10,1011.0,4.50,100,75,20.0,...,0,0,0,0,0,0,0,0,0,0
3,D0004,17.0,0,6.1,150,1018.0,0.00,88,0,17.0,...,0,0,0,0,0,0,0,0,0,0
6,D0007,21.0,1,4.0,310,1015.0,0.00,100,50,21.0,...,0,0,0,0,0,0,0,0,0,0
18,D0019,19.0,0,3.6,64,1017.0,0.00,88,0,19.0,...,0,0,0,0,0,0,0,0,0,0
27,D0028,19.0,1,3.6,83,1010.0,0.00,73,25,19.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,D2839,27.0,1,19.1,90,1014.0,0.01,70,25,29.8,...,0,0,0,0,0,0,0,1,0,0
2863,D2864,21.4,2,3.6,178,1012.0,0.00,89,75,21.4,...,0,0,0,0,0,0,0,0,1,0
2873,D2874,16.9,0,16.6,68,1018.0,0.00,44,2,16.9,...,0,0,0,0,0,0,0,0,0,1
2886,D2887,19.5,0,5.4,27,1014.0,0.00,34,0,19.4,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
'''
# Step 3: Decode the integers back into weather conditions
reverse_weather_mapping = {v: k for k, v in weather_mapping.items()}
df['weather_decoded'] = df['weather_encoded'].map(reverse_weather_mapping)
print("Decoded DataFrame:\n", df)
'''

'\n# Step 3: Decode the integers back into weather conditions\nreverse_weather_mapping = {v: k for k, v in weather_mapping.items()}\ndf[\'weather_decoded\'] = df[\'weather_encoded\'].map(reverse_weather_mapping)\nprint("Decoded DataFrame:\n", df)\n'

In [ ]:
#Save file
df.to_csv('weather_data_processed.csv', index=False)

# Feature Engineering => From HERE

In [ ]:
import pandas as pd

#df = pd.read_csv('weather_data_processed.csv', sep=',')
print(df.head())

NameError: name 'df' is not defined

In [ ]:
df = df.drop(columns=['day_id'])

In [ ]:
df.dtypes

temperature_celsius    float64
condition_text           int64
wind_kph               float64
wind_degree              int64
pressure_mb            float64
                        ...   
city_id_C108             int64
city_id_C109             int64
city_id_C110             int64
city_id_C111             int64
city_id_C112             int64
Length: 127, dtype: object

## <----------------------Base Line Changes-------------------->

In [ ]:
df = df.drop(columns=['wind_degree'])    #'wind_degree'
df['temp_diff'] = df['feels_like_celsius']-df['temperature_celsius']
#------------------------------------------------------------------Confirmed ^^
df = df.drop(columns=['air_quality_us-epa-index'])
df

,temperature_celsius,condition_text,wind_kph,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,...,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff
0,20.0,6,3.6,1011.0,4.50,100,75,20.0,10.0,1.0,...,0,0,0,0,0,0,0,0,0,0.0
1,17.0,0,6.1,1018.0,0.00,88,0,17.0,10.0,1.0,...,0,0,0,0,0,0,0,0,0,0.0
2,21.0,1,4.0,1015.0,0.00,100,50,21.0,10.0,1.0,...,0,0,0,0,0,0,0,0,0,0.0
3,19.0,0,3.6,1017.0,0.00,88,0,19.0,10.0,1.0,...,0,0,0,0,0,0,0,0,0,0.0
4,19.0,1,3.6,1010.0,0.00,73,25,19.0,10.0,1.0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,27.0,1,19.1,1014.0,0.01,70,25,29.8,10.0,6.0,...,0,0,0,0,0,0,1,0,0,2.8
475,21.4,2,3.6,1012.0,0.00,89,75,21.4,10.0,6.0,...,0,0,0,0,0,0,0,1,0,0.0
476,16.9,0,16.6,1018.0,0.00,44,2,16.9,10.0,1.0,...,0,0,0,0,0,0,0,0,1,0.0
477,19.5,0,5.4,1014.0,0.00,34,0,19.4,10.0,1.0,...,0,0,0,0,0,0,0,0,1,-0.1


# Imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import re
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelBinarizer
import numpy as np


# Tensor flow

In [ ]:
# Normalizer

def z_normalize(df, exclude_column):
    # Store the means and standard deviations
    mean_std = {}

    # Create a copy of the DataFrame to avoid changing the original
    df_normalized = df.copy()

    # Iterate over each column
    for col in df.columns:
        if col not in exclude_column:
            # Calculate the mean and standard deviation
            mean = df[col].mean()
            std = df[col].std()

            # Store the mean and standard deviation
            mean_std[col] = (mean, std)

            # Normalize the column
            df_normalized[col] = (df[col] - mean) / std

    return df_normalized, mean_std


# De-normalizer
def de_normalize(df_normalized, mean_std, exclude_column):
    # Create a copy of the DataFrame to avoid changing the original
    df_denormalized = df_normalized.copy()

    # Iterate over the stored mean and standard deviation values
    for col, (mean, std) in mean_std.items():
        # De-normalize the column
        df_denormalized[col] = df_normalized[col] * std + mean

    return df_denormalized

In [ ]:
#drop_columns = list(df.columns[-112:])
drop_columns = ['city_id_C001', 'city_id_C002', 'city_id_C003', 'city_id_C004', 'city_id_C005', 'city_id_C006',
                'city_id_C007', 'city_id_C008', 'city_id_C009', 'city_id_C010', 'city_id_C011', 'city_id_C012',
                'city_id_C013', 'city_id_C014', 'city_id_C015', 'city_id_C016', 'city_id_C017', 'city_id_C018',
                'city_id_C019', 'city_id_C020', 'city_id_C021', 'city_id_C022', 'city_id_C023', 'city_id_C024',
                'city_id_C025', 'city_id_C026', 'city_id_C027', 'city_id_C028', 'city_id_C029', 'city_id_C030',
                'city_id_C031', 'city_id_C032', 'city_id_C033', 'city_id_C034', 'city_id_C035', 'city_id_C036',
                'city_id_C037', 'city_id_C038', 'city_id_C039', 'city_id_C040', 'city_id_C041', 'city_id_C042',
                'city_id_C043', 'city_id_C044', 'city_id_C045', 'city_id_C046', 'city_id_C047', 'city_id_C048',
                'city_id_C049', 'city_id_C050', 'city_id_C051', 'city_id_C052', 'city_id_C053', 'city_id_C054',
                'city_id_C055', 'city_id_C056', 'city_id_C057', 'city_id_C058', 'city_id_C059', 'city_id_C060',
                'city_id_C061', 'city_id_C062', 'city_id_C063', 'city_id_C064', 'city_id_C065', 'city_id_C066',
                'city_id_C067', 'city_id_C068', 'city_id_C069', 'city_id_C070', 'city_id_C071', 'city_id_C072',
                'city_id_C073', 'city_id_C074', 'city_id_C075', 'city_id_C076', 'city_id_C077', 'city_id_C078',
                'city_id_C079', 'city_id_C080', 'city_id_C081', 'city_id_C082', 'city_id_C083', 'city_id_C084',
                'city_id_C085', 'city_id_C086', 'city_id_C087', 'city_id_C088', 'city_id_C089', 'city_id_C090',
                'city_id_C091', 'city_id_C092', 'city_id_C093', 'city_id_C094', 'city_id_C095', 'city_id_C096',
                'city_id_C097', 'city_id_C098', 'city_id_C099', 'city_id_C100', 'city_id_C101', 'city_id_C102',
                'city_id_C103', 'city_id_C104', 'city_id_C105', 'city_id_C106', 'city_id_C107', 'city_id_C108',
                'city_id_C109', 'city_id_C110', 'city_id_C111', 'city_id_C112','condition_text']


In [ ]:
# Make a copy for tensorflow usage
df_tensorflow = df.copy()

#normalize data
df_tensorflow_normalized, mean_std = z_normalize(df_tensorflow, exclude_column=drop_columns)

df_tensorflow_normalized

,temperature_celsius,condition_text,wind_kph,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,...,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff
0,-0.325347,6,-0.975356,-0.345452,3.804144,1.159400,0.801476,-0.441783,0.187377,-0.559426,...,0,0,0,0,0,0,0,0,0,-0.646009
1,-0.790360,0,-0.634248,0.801381,-0.246356,0.505659,-1.267732,-0.790444,0.187377,-0.559426,...,0,0,0,0,0,0,0,0,0,-0.646009
2,-0.170343,1,-0.920779,0.309881,-0.246356,1.159400,0.111740,-0.325562,0.187377,-0.559426,...,0,0,0,0,0,0,0,0,0,-0.646009
3,-0.480351,0,-0.975356,0.637547,-0.246356,0.505659,-1.267732,-0.558003,0.187377,-0.559426,...,0,0,0,0,0,0,0,0,0,-0.646009
4,-0.480351,1,-0.975356,-0.509285,-0.246356,-0.311516,-0.577996,-0.558003,0.187377,-0.559426,...,0,0,0,0,0,0,0,0,0,-0.646009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,0.759683,1,1.139515,0.146048,-0.237355,-0.474951,-0.577996,0.697176,0.187377,1.742678,...,0,0,0,0,0,0,1,0,0,0.416570
475,-0.108341,2,-0.975356,-0.181619,-0.246356,0.560138,0.801476,-0.279074,0.187377,1.742678,...,0,0,0,0,0,0,0,1,0,-0.646009
476,-0.805860,0,0.798407,0.801381,-0.246356,-1.891389,-1.212553,-0.802066,0.187377,-0.559426,...,0,0,0,0,0,0,0,0,1,-0.646009
477,-0.402849,0,-0.729758,0.146048,-0.246356,-2.436173,-1.267732,-0.511515,0.187377,-0.559426,...,0,0,0,0,0,0,0,0,1,-0.683958


In [ ]:
X = df_tensorflow_normalized.drop(columns=['condition_text']).copy()
y = df_tensorflow_normalized['condition_text'].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
# Encode the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [ ]:
def create_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
# Create the model
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]
model = create_model(input_dim, output_dim)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
10/10 [==============================] - 0s 19ms/step - loss: 2.3501 - accuracy: 0.1373 - val_loss: 2.1674 - val_accuracy: 0.1429
Epoch 2/50
10/10 [==============================] - 0s 8ms/step - loss: 2.1829 - accuracy: 0.2157 - val_loss: 2.1460 - val_accuracy: 0.1558
Epoch 3/50
10/10 [==============================] - 0s 7ms/step - loss: 2.1742 - accuracy: 0.2092 - val_loss: 2.1267 - val_accuracy: 0.1558
Epoch 4/50
10/10 [==============================] - 0s 7ms/step - loss: 2.1575 - accuracy: 0.2222 - val_loss: 2.1081 - val_accuracy: 0.1558
Epoch 5/50
10/10 [==============================] - 0s 7ms/step - loss: 2.1409 - accuracy: 0.1667 - val_loss: 2.0887 - val_accuracy: 0.1818
Epoch 6/50
10/10 [==============================] - 0s 7ms/step - loss: 2.1024 - accuracy: 0.2549 - val_loss: 2.0709 - val_accuracy: 0.1818
Epoch 7/50
10/10 [==============================] - 0s 7ms/step - loss: 2.0517 - accuracy: 0.2484 - val_loss: 2.0534 - val_accuracy: 0.1818
Epoch 8/50
10/10 [=

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

3/3 [==============================] - 0s 4ms/step - loss: 1.2388 - accuracy: 0.6250
Test Accuracy: 62.50%


## Results

Base -> 61.46% <br>
Degree dropped -> 65.62% <br>
Pressure Dropped -> 62.50% <br>   ---------- Both dropped (62.5) <br>

temp-difference ->64.58% <br>
temp-difference+degree dropped ->65.62%  <br>
+UV_index-dropped ->61.46% <br>


In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


3/3 [==============================] - 0s 2ms/step


In [ ]:
y_test_classes

array([1, 3, 1, 2, 0, 0, 1, 7, 0, 4, 1, 2, 2, 4, 1, 7, 3, 7, 0, 0, 1, 0,
       2, 1, 0, 2, 4, 1, 0, 1, 0, 3, 1, 4, 5, 1, 2, 1, 0, 1, 2, 0, 2, 0,
       7, 1, 1, 8, 4, 1, 4, 0, 4, 3, 4, 0, 0, 0, 0, 3, 1, 2, 1, 3, 0, 8,
       0, 2, 7, 4, 3, 0, 2, 0, 4, 1, 1, 0, 0, 1, 0, 3, 3, 1, 0, 8, 1, 1,
       0, 0, 2, 1, 1, 0, 1, 0], dtype=int64)

In [ ]:
y_pred_classes

array([0, 3, 1, 2, 0, 0, 2, 1, 0, 0, 1, 0, 3, 4, 1, 1, 2, 2, 0, 0, 1, 0,
       1, 3, 0, 3, 4, 2, 0, 0, 0, 3, 0, 4, 2, 2, 2, 1, 0, 1, 2, 0, 3, 0,
       1, 1, 2, 1, 4, 1, 2, 0, 0, 3, 4, 0, 0, 0, 0, 2, 2, 2, 0, 3, 0, 1,
       0, 2, 2, 4, 2, 0, 3, 0, 3, 2, 1, 0, 0, 3, 0, 3, 3, 1, 0, 2, 1, 1,
       0, 0, 1, 1, 1, 0, 2, 0], dtype=int64)

# XGBoost Model

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
drive.mount('/content/drive')
data = h2o.import_file('/content/drive/MyDrive/Kaggle/weather_data_processed.csv')

KeyboardInterrupt: 

In [ ]:
csv_file = 'weather_data_processed.csv'#make sure to have the Labels column
# csv_file = 'csv_all.csv'
df = pd.read_csv(csv_file)
print(df.head())

  day_id  temperature_celsius  condition_text  wind_kph  wind_degree  \
0  D0003                 20.0               6       3.6           10   
1  D0004                 17.0               0       6.1          150   
2  D0007                 21.0               1       4.0          310   
3  D0019                 19.0               0       3.6           64   
4  D0028                 19.0               1       3.6           83   

   pressure_mb  precip_mm  humidity  cloud  feels_like_celsius  ...  \
0       1011.0        4.5       100     75                20.0  ...   
1       1018.0        0.0        88      0                17.0  ...   
2       1015.0        0.0       100     50                21.0  ...   
3       1017.0        0.0        88      0                19.0  ...   
4       1010.0        0.0        73     25                19.0  ...   

   city_id_C103  city_id_C104  city_id_C105  city_id_C106  city_id_C107  \
0             0             0             0             0        

In [ ]:
df=df.drop(columns=['day_id'])


In [ ]:
df = df.drop(columns=['wind_degree'])    #'wind_degree'
df['temp_diff'] = df['feels_like_celsius']-df['temperature_celsius']
#------------------------------------------------------------------Confirmed ^^
df = df.drop(columns=['air_quality_us-epa-index'])
df['wind_diff'] = df['gust_kph']-df['wind_kph']
#df=df.drop(columns=['day_id'])
#df=df.drop(columns=['wind_kph'])


In [ ]:
df=df.drop(columns=['gust_kph'])
df=df.drop(columns=['wind_kph'])

In [ ]:
df=df.drop(columns=['feels_like_celsius'])

In [ ]:
# Make a copy for tensorflow usage
df_xgb = df.copy()
print(df_xgb.head())

#normalizing



   temperature_celsius  condition_text  pressure_mb  precip_mm  humidity  \
0                 20.0               6       1011.0        4.5       100   
1                 17.0               0       1018.0        0.0        88   
2                 21.0               1       1015.0        0.0       100   
3                 19.0               0       1017.0        0.0        88   
4                 19.0               1       1010.0        0.0        73   

   cloud  visibility_km  uv_index  sunrise  sunset  ...  city_id_C105  \
0     75           10.0       1.0      365    1158  ...             0   
1      0           10.0       1.0      366    1156  ...             0   
2     50           10.0       1.0      368    1151  ...             0   
3      0           10.0       1.0      380    1131  ...             0   
4     25           10.0       1.0      377    1160  ...             0   

   city_id_C106  city_id_C107  city_id_C108  city_id_C109  city_id_C110  \
0             0             0

In [ ]:
X = df_xgb.drop(columns=['condition_text']).copy()
y = df_xgb['condition_text'].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Example DataFrame df_xgb
# Assuming df_xgb already contains the relevant columns
# Replace this with your actual data loading process
# df_xgb = pd.read_csv('your_file.csv')

# Selecting only the columns to normalize
columns_to_normalize = ['temperature_celsius', 'pressure_mb', 'precip_mm',
                        'humidity', 'cloud',
                        'visibility_km', 'uv_index']

# Extracting the columns to normalize
df_to_normalize = df_xgb[columns_to_normalize]

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data to normalize it
normalized_data = scaler.fit_transform(df_to_normalize)

# Creating a DataFrame of normalized data
df_normalized = pd.DataFrame(normalized_data, columns=df_to_normalize.columns)

# Updating original DataFrame with normalized values
df_xgb[columns_to_normalize] = df_normalized

# Displaying the normalized DataFrame
print(df_xgb.head())

   temperature_celsius  condition_text  pressure_mb  precip_mm  humidity  \
0            -0.325687               6    -0.345813   3.808121  1.160612   
1            -0.791186               0     0.802218  -0.246614  0.506188   
2            -0.170521               1     0.310205  -0.246614  1.160612   
3            -0.480854               0     0.638214  -0.246614  0.506188   
4            -0.480854               1    -0.509818  -0.246614 -0.311842   

      cloud  visibility_km  uv_index  sunrise  sunset  ...  city_id_C105  \
0  0.802314       0.187573 -0.560011      365    1158  ...             0   
1 -1.269057       0.187573 -0.560011      366    1156  ...             0   
2  0.111857       0.187573 -0.560011      368    1151  ...             0   
3 -1.269057       0.187573 -0.560011      380    1131  ...             0   
4 -0.578600       0.187573 -0.560011      377    1160  ...             0   

   city_id_C106  city_id_C107  city_id_C108  city_id_C109  city_id_C110  \
0          

In [ ]:
X = df_xgb.drop(columns=['condition_text']).copy()
y = df_xgb['condition_text'].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [ ]:
param_grid = {
    'n_estimators': [20,10],
    'max_depth': [30,15],
    'learning_rate': [0.01,0.05],
    'subsample': [0.7,0.8],
    'colsample_bytree': [0.8,0.7],
    'gamma': [0,]
}


In [ ]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid,
                           scoring='accuracy', cv=3, verbose=1, n_jobs=-1)


In [ ]:
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 32 candidates, totalling 96 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.7], 'gamma': [0],
                         'learning_rate': [0.01, 0.05], 'max_depth': [30, 15],
                         'n_estimators': [20, 10], 'subsample': [0.7, 0.8]},
             scoring='accuracy', verbose=1)

In [ ]:
#drop air quality,gust = 0.6979166666666666
##drop air quality,gustkph,windkph = 0.6875
##drop air quality,gustkph,windkph,feel like temp,dif between gust and wind added, non normalzed and normalized = 0.71875
#0.70833

print(f'Best parameters found: {grid_search.best_params_}')

# Use the best model to make predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Best parameters found: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 30, 'n_estimators': 10, 'subsample': 0.7}
Accuracy: 0.6770833333333334
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93        29
           1       0.74      0.85      0.79        27
           2       0.29      0.33      0.31        12
           3       0.40      0.44      0.42         9
           4       0.60      0.60      0.60        10
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         3

    accuracy                           0.68        96
   macro avg       0.37      0.40      0.38        96
weighted avg       0.62      0.68      0.65        96



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred

array([1, 2, 1, 2, 0, 0, 1, 2, 0, 0, 1, 0, 3, 4, 1, 1, 3, 1, 0, 0, 1, 0,
       4, 1, 0, 3, 4, 1, 0, 1, 0, 3, 4, 4, 1, 1, 2, 1, 0, 1, 2, 0, 1, 0,
       1, 1, 1, 2, 4, 1, 2, 0, 0, 2, 4, 0, 0, 0, 0, 2, 2, 3, 4, 1, 0, 2,
       0, 2, 3, 4, 3, 0, 3, 0, 1, 2, 1, 0, 0, 1, 4, 2, 3, 1, 0, 3, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0])

In [ ]:
df = pd.read_csv('daily_data.csv_processed.csv', sep = ',')
df

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
0,D0001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,...,0,0,0,0,0,0,0,0,0,0
1,D0002,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,...,0,0,0,0,0,0,0,0,0,0
2,D0003,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,...,0,0,0,0,0,0,0,0,0,0
3,D0004,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,...,0,0,0,0,0,0,0,0,0,0
4,D0005,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,Clear and Sunny,13.3,61,1017.0,0.0,57,3,17.1,...,0,0,0,0,0,0,0,0,0,1
2889,D2890,17.4,NaN,13.0,51,1017.0,0.0,49,0,17.4,...,0,0,0,0,0,0,0,0,0,1
2890,D2891,19.2,NaN,11.5,46,1016.0,0.0,34,0,19.2,...,0,0,0,0,0,0,0,0,0,1
2891,D2892,19.2,NaN,14.4,76,1017.0,0.0,45,2,19.2,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df = df.drop(columns=['wind_degree'])    #'wind_degree'
df['temp_diff'] = df['feels_like_celsius']-df['temperature_celsius']
#------------------------------------------------------------------Confirmed ^^
df = df.drop(columns=['air_quality_us-epa-index'])
df['wind_diff'] = df['gust_kph']-df['wind_kph']
#df=df.drop(columns=['day_id'])
#df=df.drop(columns=['wind_kph'])
df=df.drop(columns=['gust_kph'])
df=df.drop(columns=['wind_kph'])
df=df.drop(columns=['feels_like_celsius'])

In [ ]:
weather_mapping = {
    'Clear and Sunny': 0,
    'Partly Cloudy': 1,
    'Light Precipitation': 2,
    'Cloudy and Overcast': 3,
    'Mist or Fog': 4,
    'Rain Showers': 5,
    'Light Rain with Thunder': 6,
    'Thunderstorms': 7,
    'Moderate to Heavy Rain': 8
}
df['condition_text'] = df['condition_text'].map(weather_mapping)
df

,day_id,temperature_celsius,condition_text,pressure_mb,precip_mm,humidity,cloud,visibility_km,uv_index,sunrise,...,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff,wind_diff
0,D0001,27.0,NaN,1006.0,0.0,54,75,10.0,6.0,364,...,0,0,0,0,0,0,0,0,1.0,5.8
1,D0002,22.0,NaN,1006.0,0.0,73,75,10.0,1.0,365,...,0,0,0,0,0,0,0,0,2.5,17.3
2,D0003,20.0,6.0,1011.0,4.5,100,75,10.0,1.0,365,...,0,0,0,0,0,0,0,0,0.0,9.0
3,D0004,17.0,0.0,1018.0,0.0,88,0,10.0,1.0,366,...,0,0,0,0,0,0,0,0,0.0,5.1
4,D0005,18.0,NaN,1019.0,0.0,94,0,10.0,1.0,367,...,0,0,0,0,0,0,0,0,0.0,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,0.0,1017.0,0.0,57,3,10.0,1.0,349,...,0,0,0,0,0,0,0,1,0.0,8.9
2889,D2890,17.4,NaN,1017.0,0.0,49,0,10.0,1.0,349,...,0,0,0,0,0,0,0,1,0.0,9.2
2890,D2891,19.2,NaN,1016.0,0.0,34,0,10.0,1.0,348,...,0,0,0,0,0,0,0,1,0.0,9.8
2891,D2892,19.2,NaN,1017.0,0.0,45,2,10.0,1.0,347,...,0,0,0,0,0,0,0,1,0.0,10.5


In [ ]:
df['condition_text']=df['condition_text'].astype('Int64')
df

,day_id,temperature_celsius,condition_text,pressure_mb,precip_mm,humidity,cloud,visibility_km,uv_index,sunrise,...,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff,wind_diff
0,D0001,27.0,<NA>,1006.0,0.0,54,75,10.0,6.0,364,...,0,0,0,0,0,0,0,0,1.0,5.8
1,D0002,22.0,<NA>,1006.0,0.0,73,75,10.0,1.0,365,...,0,0,0,0,0,0,0,0,2.5,17.3
2,D0003,20.0,6,1011.0,4.5,100,75,10.0,1.0,365,...,0,0,0,0,0,0,0,0,0.0,9.0
3,D0004,17.0,0,1018.0,0.0,88,0,10.0,1.0,366,...,0,0,0,0,0,0,0,0,0.0,5.1
4,D0005,18.0,<NA>,1019.0,0.0,94,0,10.0,1.0,367,...,0,0,0,0,0,0,0,0,0.0,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,0,1017.0,0.0,57,3,10.0,1.0,349,...,0,0,0,0,0,0,0,1,0.0,8.9
2889,D2890,17.4,<NA>,1017.0,0.0,49,0,10.0,1.0,349,...,0,0,0,0,0,0,0,1,0.0,9.2
2890,D2891,19.2,<NA>,1016.0,0.0,34,0,10.0,1.0,348,...,0,0,0,0,0,0,0,1,0.0,9.8
2891,D2892,19.2,<NA>,1017.0,0.0,45,2,10.0,1.0,347,...,0,0,0,0,0,0,0,1,0.0,10.5


In [ ]:
# Predict on the rows where target is NaN
X_pred = df[df['condition_text'].isna()].drop('condition_text', axis=1).drop(columns=['day_id'])
predictions = best_model.predict(X_pred)

# Fill NaN values with predictions
df.loc[df['condition_text'].isna(), 'condition_text'] = predictions
df

,day_id,temperature_celsius,condition_text,pressure_mb,precip_mm,humidity,cloud,visibility_km,uv_index,sunrise,...,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff,wind_diff
0,D0001,27.0,3,1006.0,0.0,54,75,10.0,6.0,364,...,0,0,0,0,0,0,0,0,1.0,5.8
1,D0002,22.0,3,1006.0,0.0,73,75,10.0,1.0,365,...,0,0,0,0,0,0,0,0,2.5,17.3
2,D0003,20.0,6,1011.0,4.5,100,75,10.0,1.0,365,...,0,0,0,0,0,0,0,0,0.0,9.0
3,D0004,17.0,0,1018.0,0.0,88,0,10.0,1.0,366,...,0,0,0,0,0,0,0,0,0.0,5.1
4,D0005,18.0,1,1019.0,0.0,94,0,10.0,1.0,367,...,0,0,0,0,0,0,0,0,0.0,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,0,1017.0,0.0,57,3,10.0,1.0,349,...,0,0,0,0,0,0,0,1,0.0,8.9
2889,D2890,17.4,1,1017.0,0.0,49,0,10.0,1.0,349,...,0,0,0,0,0,0,0,1,0.0,9.2
2890,D2891,19.2,1,1016.0,0.0,34,0,10.0,1.0,348,...,0,0,0,0,0,0,0,1,0.0,9.8
2891,D2892,19.2,3,1017.0,0.0,45,2,10.0,1.0,347,...,0,0,0,0,0,0,0,1,0.0,10.5


In [ ]:
submission = df[['day_id','condition_text']]
submission

,day_id,condition_text
0,D0001,3
1,D0002,3
2,D0003,6
3,D0004,0
4,D0005,1
...,...,...
2888,D2889,0
2889,D2890,1
2890,D2891,1
2891,D2892,3


In [ ]:
reverse_weather_mapping = {v: k for k, v in weather_mapping.items()}
submission['condition_text'] = submission['condition_text'].map(reverse_weather_mapping)
submission.to_csv('submission_XGB_full_set.csv', index=False)
submission

<ipython-input-64-e77d6cd59ea2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['condition_text'] = submission['condition_text'].map(reverse_weather_mapping)


,day_id,condition_text
0,D0001,Cloudy and Overcast
1,D0002,Cloudy and Overcast
2,D0003,Light Rain with Thunder
3,D0004,Clear and Sunny
4,D0005,Partly Cloudy
...,...,...
2888,D2889,Clear and Sunny
2889,D2890,Partly Cloudy
2890,D2891,Partly Cloudy
2891,D2892,Cloudy and Overcast


In [ ]:
#drop air quality

print(f'Best parameters found: {grid_search.best_params_}')

# Use the best model to make predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Best parameters found: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 250, 'subsample': 0.6}
Accuracy: 0.6979166666666666
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93        29
           1       0.71      0.93      0.81        27
           2       0.25      0.25      0.25        12
           3       0.50      0.56      0.53         9
           4       0.75      0.60      0.67        10
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         3

    accuracy                           0.70        96
   macro avg       0.39      0.41      0.40        96
weighted avg       0.63      0.70      0.66        96



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f'Best parameters found: {grid_search.best_params_}')

# Use the best model to make predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Best parameters found: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
Accuracy: 0.6770833333333334
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93        29
           1       0.71      0.93      0.81        27
           2       0.21      0.25      0.23        12
           3       0.38      0.33      0.35         9
           4       0.75      0.60      0.67        10
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         3

    accuracy                           0.68        96
   macro avg       0.37      0.38      0.37        96
weighted avg       0.61      0.68      0.64        96



C:\Users\Panduka\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Panduka\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Panduka\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Predictions

In [ ]:
import pandas as pd

df = pd.read_csv('daily_data.csv_processed.csv', sep = ',')
df

,day_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
0,D0001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,...,0,0,0,0,0,0,0,0,0,0
1,D0002,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,...,0,0,0,0,0,0,0,0,0,0
2,D0003,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,...,0,0,0,0,0,0,0,0,0,0
3,D0004,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,...,0,0,0,0,0,0,0,0,0,0
4,D0005,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,Clear and Sunny,13.3,61,1017.0,0.0,57,3,17.1,...,0,0,0,0,0,0,0,0,0,1
2889,D2890,17.4,NaN,13.0,51,1017.0,0.0,49,0,17.4,...,0,0,0,0,0,0,0,0,0,1
2890,D2891,19.2,NaN,11.5,46,1016.0,0.0,34,0,19.2,...,0,0,0,0,0,0,0,0,0,1
2891,D2892,19.2,NaN,14.4,76,1017.0,0.0,45,2,19.2,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df = df.drop(columns=['wind_degree'])    #'wind_degree'
df['temp_diff'] = df['feels_like_celsius']-df['temperature_celsius']
df

,day_id,temperature_celsius,condition_text,wind_kph,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,...,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff
0,D0001,27.0,NaN,6.1,1006.0,0.0,54,75,28.0,10.0,...,0,0,0,0,0,0,0,0,0,1.0
1,D0002,22.0,NaN,6.1,1006.0,0.0,73,75,24.5,10.0,...,0,0,0,0,0,0,0,0,0,2.5
2,D0003,20.0,Light Rain with Thunder,3.6,1011.0,4.5,100,75,20.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
3,D0004,17.0,Clear and Sunny,6.1,1018.0,0.0,88,0,17.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
4,D0005,18.0,NaN,3.6,1019.0,0.0,94,0,18.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,Clear and Sunny,13.3,1017.0,0.0,57,3,17.1,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2889,D2890,17.4,NaN,13.0,1017.0,0.0,49,0,17.4,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2890,D2891,19.2,NaN,11.5,1016.0,0.0,34,0,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2891,D2892,19.2,NaN,14.4,1017.0,0.0,45,2,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0


In [ ]:
weather_mapping = {
    'Clear and Sunny': 0,
    'Partly Cloudy': 1,
    'Light Precipitation': 2,
    'Cloudy and Overcast': 3,
    'Mist or Fog': 4,
    'Rain Showers': 5,
    'Light Rain with Thunder': 6,
    'Thunderstorms': 7,
    'Moderate to Heavy Rain': 8
}
df['condition_text'] = df['condition_text'].map(weather_mapping)
df

,day_id,temperature_celsius,condition_text,wind_kph,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,...,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff
0,D0001,27.0,NaN,6.1,1006.0,0.0,54,75,28.0,10.0,...,0,0,0,0,0,0,0,0,0,1.0
1,D0002,22.0,NaN,6.1,1006.0,0.0,73,75,24.5,10.0,...,0,0,0,0,0,0,0,0,0,2.5
2,D0003,20.0,6.0,3.6,1011.0,4.5,100,75,20.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
3,D0004,17.0,0.0,6.1,1018.0,0.0,88,0,17.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
4,D0005,18.0,NaN,3.6,1019.0,0.0,94,0,18.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,0.0,13.3,1017.0,0.0,57,3,17.1,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2889,D2890,17.4,NaN,13.0,1017.0,0.0,49,0,17.4,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2890,D2891,19.2,NaN,11.5,1016.0,0.0,34,0,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2891,D2892,19.2,NaN,14.4,1017.0,0.0,45,2,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0


In [ ]:
df['condition_text']=df['condition_text'].astype('Int64')
df

,day_id,temperature_celsius,condition_text,wind_kph,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,...,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff
0,D0001,27.0,<NA>,6.1,1006.0,0.0,54,75,28.0,10.0,...,0,0,0,0,0,0,0,0,0,1.0
1,D0002,22.0,<NA>,6.1,1006.0,0.0,73,75,24.5,10.0,...,0,0,0,0,0,0,0,0,0,2.5
2,D0003,20.0,6,3.6,1011.0,4.5,100,75,20.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
3,D0004,17.0,0,6.1,1018.0,0.0,88,0,17.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
4,D0005,18.0,<NA>,3.6,1019.0,0.0,94,0,18.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,0,13.3,1017.0,0.0,57,3,17.1,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2889,D2890,17.4,<NA>,13.0,1017.0,0.0,49,0,17.4,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2890,D2891,19.2,<NA>,11.5,1016.0,0.0,34,0,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2891,D2892,19.2,<NA>,14.4,1017.0,0.0,45,2,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0


In [ ]:
# Predict on the rows where target is NaN
X_pred = df[df['condition_text'].isna()].drop('condition_text', axis=1).drop(columns=['day_id'])
predictions = best_model.predict(X_pred)

# Fill NaN values with predictions
df.loc[df['condition_text'].isna(), 'condition_text'] = predictions
df

,day_id,temperature_celsius,condition_text,wind_kph,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,...,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,temp_diff
0,D0001,27.0,1,6.1,1006.0,0.0,54,75,28.0,10.0,...,0,0,0,0,0,0,0,0,0,1.0
1,D0002,22.0,1,6.1,1006.0,0.0,73,75,24.5,10.0,...,0,0,0,0,0,0,0,0,0,2.5
2,D0003,20.0,6,3.6,1011.0,4.5,100,75,20.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
3,D0004,17.0,0,6.1,1018.0,0.0,88,0,17.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
4,D0005,18.0,0,3.6,1019.0,0.0,94,0,18.0,10.0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,17.1,0,13.3,1017.0,0.0,57,3,17.1,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2889,D2890,17.4,0,13.0,1017.0,0.0,49,0,17.4,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2890,D2891,19.2,0,11.5,1016.0,0.0,34,0,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0
2891,D2892,19.2,0,14.4,1017.0,0.0,45,2,19.2,10.0,...,0,0,0,0,0,0,0,0,1,0.0


In [ ]:
submission = df[['day_id','condition_text']]
submission

,day_id,condition_text
0,D0001,1
1,D0002,1
2,D0003,6
3,D0004,0
4,D0005,0
...,...,...
2888,D2889,0
2889,D2890,0
2890,D2891,0
2891,D2892,0


In [ ]:
reverse_weather_mapping = {v: k for k, v in weather_mapping.items()}
submission['condition_text'] = submission['condition_text'].map(reverse_weather_mapping)
submission.to_csv('submission_XGB.csv', index=False)
submission


C:\Users\Panduka\AppData\Local\Temp\ipykernel_5148\1706578027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['condition_text'] = submission['condition_text'].map(reverse_weather_mapping)


,day_id,condition_text
0,D0001,Partly Cloudy
1,D0002,Partly Cloudy
2,D0003,Light Rain with Thunder
3,D0004,Clear and Sunny
4,D0005,Clear and Sunny
...,...,...
2888,D2889,Clear and Sunny
2889,D2890,Clear and Sunny
2890,D2891,Clear and Sunny
2891,D2892,Clear and Sunny


# CatBoost